In [2]:
from requests import get

# https://stackoverflow.com/questions/7243750/download-file-from-web-in-python-3
def download(url, file_name):
    # open in binary mode
    with open(file_name, "wb") as file:
        # get request
        response = get(url)
        # write to file
        file.write(response.content)
        return file.name

total_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_000044.6&rettype=fasta"
cds_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_000044.6&rettype=fasta_cds_na"

total_gene = download(total_url, "total_gene.fasta")
cds_gene = download(cds_url, "cds.fasta")

#potentially could remove/slim this function down if we do not need the fasta files themselves, could directly translate to text

In [3]:
#open both fasta and cds files
total_txt = ""
cds_txt = ""

with open(total_gene, "r") as total_fasta:
    total_txt = total_fasta.read() # STRING

with open(cds_gene, "r") as cds_fasta:
    cds_txt = cds_fasta.read()


In [4]:
#Assuming that the header of fasta files contain > and are broken by lines:
def split_trim_fasta(fasta_txt):
    split_fasta = [line for line in fasta_txt.split("\n") if line.strip()]       
    nuc_only = ''.join([c for c in split_fasta if (c[0] != ">")])

    return nuc_only

total_nuc = split_trim_fasta(total_txt)
cds_nuc = split_trim_fasta(cds_txt)

In [5]:
print(total_nuc)

AGCGCCCCCTCCGAGATCCCGGGGAGCCAGCTTGCTGGGAGAGCGGGACGGTCCGGAGCAAGCCCAGAGGCAGAGGAGGCGACAGAGGGAAAAAGGGCCGAGCTAGCCGCTCCAGTGCTGTACAGGAGCCGAAGGGACGCACCACGCCAGCCCCAGCCCGGCTCCAGCGACAGCCAACGCCTCTTGCAGCGCGGCGGCTTCGAAGCCGCCGCCCGGAGCTGCCCTTTCCTCTTCGGTGAAGTTTTTAAAAGCTGCTAAAGACTCGGAGGAAGCAAGGAAAGTGCCTGGTAGGACTGACGGCTGCCTTTGTCCTCCTCCTCTCCACCCCGCCTCCCCCCACCCTGCCTTCCCCCCCTCCCCCGTCTTCTCTCCCGCAGCTGCCTCAGTCGGCTACTCTCAGCCAACCCCCCTCACCACCCTTCTCCCCACCCGCCCCCCCGCCCCCGTCGGCCCAGCGCTGCCAGCCCGAGTTTGCAGAGAGGTAACTCCCTTTGGCTGCGAGCGGGCGAGCTAGCTGCACATTGCAAAGAAGGCTCTTAGGAGCCAGGCGACTGGGGAGCGGCTTCAGCACTGCAGCCACGACCCGCCTGGTTAGGCTGCACGCGGAGAGAACCCTCTGTTTTCCCCCACTCTCTCTCCACCTCCTCCTGCCTTCCCCACCCCGAGTGCGGAGCCAGAGATCAAAAGATGAAAAGGCAGTCAGGTCTTCAGTAGCCAAAAAACAAAACAAACAAAAACAAAAAAGCCGAAATAAAAGAAAAAGATAATAACTCAGTTCTTATTTGCACCTACTTCAGTGGACACTGAATTTGGAAGGTGGAGGATTTTGTTTTTTTCTTTTAAGATCTGGGCATCTTTTGAATCTACCCTTCAAGTATTAAGAGACAGACTGTGAGCCTAGCAGGGCAGATCTTGTCCACCGTGTGTCTTCTTCTGCACGAGACTTTGAGGCTGTCAGAGCGCTTTTTGCGTGGTTGCTCCCGCAAGTTTCCTTCTCTGG

# Stackoverflow, my one true love (trying to query through esearch)
https://stackoverflow.com/questions/13557981/getting-a-gene-sequence-from-entrez-using-biopython

In [44]:
import time

Entrez.email = "christinelee@fas.harvard.edu"

gene = "AR"
search_string = gene+"[Gene] AND " + "Homo sapiens[Organism] AND RefSeq[Filter]"

handle = Entrez.esearch(db="nuccore", term=search_string)
record = Entrez.read(handle)
handle.close()

ids = record['IdList']
print(ids)

for id in ids:
    seq_id = ids[0] #you must implement an if to deal with <0 or >1 cases
    handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
    record = handle.read()
    handle.close()

    print(record)
    time.sleep(1)

out_handle = open('myfasta.fasta', 'w')
out_handle.write(record.rstrip('\n'))

['1890351249', '1519243240', '1070575231', '2194974292', '2194973865', '2194972658', '568815594', '568815591', '568815575', '1519245710', '2574688511', '1654124212', '1143076918', '1143076916', '1143076911', '1143076910', '350606318']
>NR_144376.2 Homo sapiens aldo-keto reductase family 1 member B (AKR1B1), transcript variant 3, non-coding RNA
GCACCGTACTGGGCGGGGGTCTGGGGAGCGCAGCAGCCATGGCAAGCCGTCTCCTGCTCAACAACGGCGC
CAAGATGCCCATCCTGGGGTTGGGTACCTGGAAGTCCCCTCCAGGGCAGGTGACTGAGGCCGTGAAGGTG
GCCATTGACGTCGGGTACCGCCACATCGACTGTGCCCATGTGTACCAGAATGAGAATGAGGTGGGGGTGG
CCATTCAGGAGAAGCTCAGGGAGCAGGTGGTGAAGCGTGAGGAGCTCTTCATCGTCAGCAAGCTGTGGTG
CACGTACCATGAGAAGGGCCTGGTGAAAGGAGCCTGCCAGAAGACACTCAGCGACCTGAAGCTGGACTAC
CTGGACCTCTACCTTATTCACTGGCCGACTGGCTTTAAGCCTGGGAAGGAATTTTTCCCATTGGATGAGT
CGGGCAATGTGGTTCCCAGTGACACCAACATTCTGGACACGTGGGCGGCCATGGAAGAGCTGGTGGATGA
AGGGCTGGTGAAAGCTATTGGCATCTCCAACTTCAACCATCTCCAGGTGGAGATGATCTTAAACAAACCT
GGCTTGAAGTATAAGCCTGCAGTTAACCAGATTGAGTGCCACCCATATCTCACTCAGGAGAAGTTAATCC
AGTACTGCCAGTCCA

2097

# Fetch Exons and CDS from NCBI

Use vars(feature) to find variables for things in biopython

**First one:** {'location': SimpleLocation(ExactPosition(0), ExactPosition(10667), strand=1), 'type': 'source', 'id': '<unknown id>', 'qualifiers': {'organism': ['Homo sapiens'], 'mol_type': ['mRNA'], 'db_xref': ['taxon:9606'], 'chromosome': ['X'], 'map': ['Xq12']}}

**CDS** {'location': SimpleLocation(ExactPosition(1126), ExactPosition(3889), strand=1), 'type': 'CDS', 'id': '<unknown id>', 'qualifiers': {'gene': ['AR'], 'gene_synonym': ['AIS; AR8; DHTR; HUMARA; HYSP1; KD; NR3C4; SBMA; SMAX1; TFM'], 'note': ['isoform 1 is encoded by transcript variant 1; dihydrotestosterone receptor; nuclear receptor subfamily 3 group C member 4'], 'codon_start': ['1'], 'product': ['androgen receptor isoform 1'], 'protein_id': ['NP_000035.2'], 'db_xref': ['CCDS:CCDS14387.1', 'GeneID:367', 'HGNC:HGNC:644', 'MIM:313700'], 'translation': ['MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAASAAPPGASLLLLQQQQQQQQQQQQQQQQQQQQQQQETSPRQQQQQQGEDGSPQAHRRGPTGYLVLDEEQQPSQPQSALECHPERGCVPEPGAAVAASKGLPQQLPAPPDEDDSAAPSTLSLLGPTFPGLSSCSADLKDILSEASTMQLLQQQQQEAVSEGSSSGRAREASGAPTSSKDNYLGGTSTISDNAKELCKAVSVSMGLGVEALEHLSPGEQLRGDCMYAPLLGVPPAVRPTPCAPLAECKGSLLDDSAGKSTEDTAEYSPFKGGYTKGLEGESLGCSGSAAAGSSGTLELPSTLSLYKSGALDEAAAYQSRDYYNFPLALAGPPPPPPPPHPHARIKLENPLDYGSAWAAAAAQCRYGDLASLHGAGAAGPGSGSPSAAASSSWHTLFTAEEGQLYGPCGGGGGGGGGGGGGGGGGGGGGGGEAGAVAPYGYTRPPQGLAGQESDFTAPDVWYPGGMVSRVPYPSPTCVKSEMGPWMDSYSGPYGDMRLETARDHVLPIDYYFPPQKTCLICGDEASGCHYGALTCGSCKVFFKRAAEGKQKYLCASRNDCTIDKFRRKNCPSCRLRKCYEAGMTLGARKLKKLGNLKLQEEGEASSTTSPTEETTQKLTVSHIEGYECQPIFLNVLEAIEPGVVCAGHDNNQPDSFAALLSSLNELGERQLVHVVKWAKALPGFRNLHVDDQMAVIQYSWMGLMVFAMGWRSFTNVNSRMLYFAPDLVFNEYRMHKSRMYSQCVRMRHLSQEFGWLQITPQEFLCMKALLLFSIIPVDGLKNQKFFDELRMNYIKELDRIIACKRKNPTSCSRRFYQLTKLLDSVQPIARELHQFTFDLLIKSHMVSVDFPEMMAEIISVQVPKILSGKVKPIYFHTQ']}}
    
**EXON** {'location': SimpleLocation(ExactPosition(2894), ExactPosition(3011), strand=1), 'type': 'exon', 'id': '<unknown id>', 'qualifiers': {'gene': ['AR'], 'gene_synonym': ['AIS; AR8; DHTR; HUMARA; HYSP1; KD; NR3C4; SBMA; SMAX1; TFM'], 'inference': ['alignment:Splign:2.1.0']}}

In [48]:
from Bio import Entrez, SeqIO

def fetch_CDS_exons(gene_id):    
    # Fetch the sequence
    handle = Entrez.efetch(db="nuccore", id=gene_id, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    
    # Collect exon positions
    exons = []
    CDS = ""
    start_flag = True
    #Run through records, searching for exons. Add exon (start, end) OR for CDS, the translation.
    for feature in record.features:
        print(feature)
        if feature.type == "exon":
            #if(len(exons) == 0):

            exons.append((int(feature.location._start), int(feature.location._end)))
        elif feature.type == "CDS":
            CDS = feature.qualifiers['translation'][0]
    print(exons)
            
    return {"exons": exons, "CDS": CDS}

#Entrez for some reason returns the entire start of the mRNA sequence 
def find_start_and_end(exon_list):
    start = "ATG"
    stop = ["TAA", "TAG", "TGA"]
    
    #find the first start codon in first exon divisible by 3:
    exon1 = exon_list[0]
    exonlast = exon_list[-1]
    
    
    print(exon1.index(start))
    print(exon1[exon1.index(start):])
    

#Given a list of tuples, full gene, and padding, generate the gene exon transcript

#currently not working wadge because this is getting only the mrna
def generate_padded_exons(exon_positions, gene, padding):
    #exons = [gene[start-padding:start].lower() + gene[start:end] + gene[end:end+padding].lower() for (start, end) in exon_positions]
    exons = [gene[start:end] for (start, end) in exon_positions]
    for i in exons:
        print(i, "\n\n\n")
        
    return exons
    

Entrez.email = "christinelee@fas.harvard.edu"
gene_id = "NM_000044.6"


#total_nuc defined above, change this
exon_positions = fetch_CDS_exons(gene_id)
exons = generate_padded_exons(exon_positions['exons'], total_nuc, 20)
start_stop = find_start_and_end(exons)

type: source
location: [0:10667](+)
qualifiers:
    Key: chromosome, Value: ['X']
    Key: db_xref, Value: ['taxon:9606']
    Key: map, Value: ['Xq12']
    Key: mol_type, Value: ['mRNA']
    Key: organism, Value: ['Homo sapiens']

type: gene
location: [0:10667](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:367', 'HGNC:HGNC:644', 'MIM:313700']
    Key: gene, Value: ['AR']
    Key: gene_synonym, Value: ['AIS; AR8; DHTR; HUMARA; HYSP1; KD; NR3C4; SBMA; SMAX1; TFM']
    Key: note, Value: ['androgen receptor']

type: exon
location: [0:2742](+)
qualifiers:
    Key: gene, Value: ['AR']
    Key: gene_synonym, Value: ['AIS; AR8; DHTR; HUMARA; HYSP1; KD; NR3C4; SBMA; SMAX1; TFM']
    Key: inference, Value: ['alignment:Splign:2.1.0']

type: misc_feature
location: [1021:1024](+)
qualifiers:
    Key: gene, Value: ['AR']
    Key: gene_synonym, Value: ['AIS; AR8; DHTR; HUMARA; HYSP1; KD; NR3C4; SBMA; SMAX1; TFM']
    Key: note, Value: ['upstream in-frame stop codon']

type: CDS
location: [1126:388